# brownian motion

Consider a European call option with parameters: T = expiry = 1, K = exercise price = 50; r = 0.1; σ = 0.3; $S_0$ = 50.

(a) Compute the Black-Scholes-Merton price of the option, i.e., the exact option price.

(b) Use random-shift Halton sequences to obtain 40 “independent” estimates for the price of the option. For each estimate, use N = 10,000 price paths. To simulate a path, you will simulate the geometric Brownian motion model with μ = r, and using 10 time steps $t_0$ = 0, $t_1$ = ∆t, $t_2$ = 2∆t, ..., $t_{10}$ = 10∆t = T.  Use the Box-Muller method to generate the standard normal numbers.  (Note that the European call option can actually be estimated by directly generating the price at expiry, so there is no need to generate the complete price path. Nevertheless, this problem prepares you for the pricing of path dependent options.)

(c) Repeat part (b) using Beasley-Springer-Moro algorithm.

(d) Compare the accuracy of the estimates you obtained in parts (b) and (c) as follows: Each set
of 40 estimates should be distributed according to the normal distribution whose mean is the
true option price you found in part (a), and an unknown variance. Apply the Anderson-Darling
statistic to test the data you obtained in parts (b) and (c), for this hypothesis. Consult Stephens’
paper on Anderson-Darling statistic for critical points. Which one of the cases discussed in the
paper applies to this problem? What are your conclusions for each data set? Which method is
better; Box-Muller or Moro?

In [55]:
# use future imports for python 3.x forward compatibility
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

# other imports
import math
import numpy
import random
from scipy.stats import anderson
import matplotlib.pyplot as plt
%matplotlib notebook

# project imports
import rand
import halton
import bfs
import box_muller as bm
import beasley_springer_moro as moro
import anderson

### Compute the Black-Scholes-Merton price of the option, i.e., the exact option price.

Reference:
John C. Hull (2003), "Options, Futures, and Other Derivatives", 5th Edition, pg. 246-249

In [56]:
def _Np(x):
    return math.exp(-0.5*x*x)/(math.sqrt(2*math.pi))

def _N(x):
    ''' N(x) is the CDF for the standard normal ditribution '''
    
    x_orig = x
    x = abs(x)

    a1 =  0.319381530;  a2 = -0.356563782;  a3 =  1.781477937;
    a4 = -1.821255978;  a5 =  1.330274429;
    
    y = 0.2316419
    k = 1/(1 + y*x);
    
    m = 1 - _Np(x)*(a1*k + a2*k**2 + a3*k**3 + a4*k**4 + a5*k**5)
    
    if x_orig < 0:
        m = 1 - m
    
    return m

def _d1(S0, K, r, sigma, T):
    return (math.log(S0/K) + (r + 0.5*sigma**2)*T)/(sigma*math.sqrt(T))
    
def _d2(S0, K, r, sigma, T):
    return (math.log(S0/K) + (r - 0.5*sigma**2)*T)/(sigma*math.sqrt(T))

def call_price(S0, K, r, sigma, T):
    ''' European call option price '''
    
    d1 = _d1(S0, K, r, sigma, T)
    d2 = _d2(S0, K, r, sigma, T)
    
    c = S0 * _N(d1) - K * math.exp(-r*T) * _N(d2)
    return c;

def put_price(S0, K, r, sigma, T):
    ''' European call option price '''
    
    d1 = _d1(S0, K, r, sigma, T)
    d2 = _d2(S0, K, r, sigma, T)
    p = K * math.exp(-r*T) * _N(-d2) - S0 * _N(-d1)
    return p;

In [57]:
# from Hull example pg. 249, compare to c=4.76, p=0.81
S0 = 42; K = 40; r = 0.1; sigma = 0.2; T = 0.5
c = call_price(S0, K, r, sigma, T)
p = put_price(S0, K, r, sigma, T)
print('call option price = {:.2f}'.format(c))
print('put  option price = {:.2f}'.format(p))

call option price = 4.76
put  option price = 0.81


### (a) Compute the Black-Scholes-Merton price of the option, i.e., the exact option price.

T = expiry = 1, K = exercise price = 50; r = 0.1; σ = 0.3; $S_0$ = 50.

check answer:
https://goodcalculators.com/black-scholes-calculator/

In [58]:
S0 = 50; K = 50; r = 0.1; sigma = 0.3; T = 1
c = call_price(S0, K, r, sigma, T)
p = put_price(S0, K, r, sigma, T)
print('call option price = {:.2f}'.format(c))
print('put  option price = {:.2f}'.format(p))

call option price = 8.37
put  option price = 3.61


### (b) geometric Brownian motion model w/ Box-Muller method
### (c) geometric Brownian motion model w/ Beasley-Springer-Moro algorithm

In [59]:
def gbm(S0, K, r, sigma, T):
    ''' QMC generation of geometric brownian motion (GBM) '''
    
    N = 10000
    n = 10
    seq = rand.rand_seq
    seq = halton.halton_seq
    seq = bfs.bfs_seq
    
    dt = T/n

    #M = bm.box_muller_seq(s=n, N=N, seq=seq)
    M = moro.beasley_springer_moro_seq(s=n, N=N, seq=seq)

    paths = []
    
    for j in range(1, N+1):
        S = [0]*(n+1)
        S[0] = S0
        for t in range(1, n+1):
            Z = M[j-1][t-1]
            S[t] = S[t - 1]*math.exp((r - 0.5*sigma**2)*dt +
                                     sigma*math.sqrt(dt)*Z)
        paths.append(S)
        
    return paths

In [60]:
def stock_price(STs):
    mean = numpy.mean(STs)
    print('stock price = {:.2f}'.format(mean))
    return mean

def call_payoff(STs):
    payoffs = [max(ST - K, 0) for ST in STs]
    mean = numpy.mean(payoffs)
    print('call option payoff = {:.2f}'.format(mean))
    return mean

def call_price(mean):
    discount_factor = math.exp(-r * T)
    mean *= discount_factor
    print('call option price = {:.2f}'.format(mean))
    return mean

In [61]:
# run the simulation 40 times
N = 40
prices = []
for i in range(N):

    print('{}--------------------------------'.format(i))
    paths = gbm(S0, K, r, sigma, T)
    STs = [path[-1] for path in paths]
    STs = numpy.array(STs)

    stock_price(STs)
    mean = call_payoff(STs)
    mean = call_price(mean)
    prices.append(mean)

0--------------------------------
stock price = 55.25
call option payoff = 9.24
call option price = 8.36
1--------------------------------
stock price = 55.31
call option payoff = 9.33
call option price = 8.44
2--------------------------------
stock price = 55.19
call option payoff = 9.14
call option price = 8.27
3--------------------------------
stock price = 55.28
call option payoff = 9.31
call option price = 8.42
4--------------------------------
stock price = 55.29
call option payoff = 9.29
call option price = 8.40
5--------------------------------
stock price = 55.29
call option payoff = 9.30
call option price = 8.42
6--------------------------------
stock price = 55.20
call option payoff = 9.12
call option price = 8.25
7--------------------------------
stock price = 55.27
call option payoff = 9.24
call option price = 8.36
8--------------------------------
stock price = 55.30
call option payoff = 9.32
call option price = 8.44
9--------------------------------
stock price = 55.27
c

In [62]:
# compute mean, variance, and standard deviation
mean = numpy.mean(prices)
var = numpy.var(prices)
std = numpy.std(prices)
exact = c
error = abs(mean - exact) / exact * 100
print('mean = {}'.format(mean))
print('variance = {}'.format(var))
print('standard deviation = {:.2e}'.format(std))
print('error = {:.2e}%'.format(error))
print('average call option price = {:.2f}'.format(mean))

mean = 8.38397498159
variance = 0.00311063485515
standard deviation = 5.58e-02
error = 2.02e-01%
average call option price = 8.38


Statistic:    
10.00: 1.760    
5.000: 2.323  
2.500: 2.904  
1.000: 3.690  

In [63]:
A = anderson.anderson_darling(prices, mean=8.37)
print('anderson darling = {}'.format(A))

anderson darling = 2.39391926521
